In [4]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import trange

import torch
from torch.utils.data import DataLoader


device = 'cpu'

In [5]:
def train(model, hp, data_loader, debug = False):
    """Train a given model for a specific type of hyperparamters and data_loader, train
    the model and then return the trained model as well as the running losses throughout 
    training.
    
    Parameters
    ----------
    model: torch.nn.Module
        An untrained model to be optimized.
    
    hp: dict
        A dictionary containing all of the hyperparameters for the system.
    
    data_loader: torch.utils.DataLoader
        A presetup dataloader containing the training data set for the set. 
    
    Returns
    -------
    model: torch.nn.Module
        The trained model after optimization.
        
    running_losses: list
        The loss for each epoch of training. 
    """
    
    # Store the losses per epoch
    running_losses = []
    
    # Configure optimizer. 
    optimizer = torch.optim.Adam(model.parameters(), lr=hp["lr"], weight_decay= hp["weight_decay"])

    criterion = torch.nn.MSELoss()
    
    # Outerloop will iterate through epochs. tqdm function trange provides progressbar
    for i in trange(hp["epochs"]):
        
        
        epoch_loss = 0 
        # Inner loop iterates through batches
        for batch in data_loader:

            # Transfer the batch to the GPU
            batch = batch.to(device)

            if debug:
                print("BATCH SHAPE: ")
                print(batch)

            # Zero gradient
            optimizer.zero_grad()

            # Perform forward pass
            pred, code = model(batch)

            # Uncomment to verify model prediction shape
            if debug:
                print("PREDI SHAPE: ")
                print(batch)

            # Compute reconstruction loss
            batch_loss = criterion(pred,batch)
            
            if debug:
                print(batch_loss)

            # Compute gradient
            batch_loss.backward()
            

            # Take step
            optimizer.step()

            # Append to running epoch loss
            epoch_loss += batch_loss.item()

        # Keep running track of losses
        if i % 250 == 0:
            print(f"Epoch [{i}]: " + str(epoch_loss))
    
        running_losses.append(epoch_loss)

    return model, running_losses

In [8]:
X = np.loadtxt("./figure_simulations.csv", delimiter = ",")

X = X.reshape(( X.shape[0], 1, X.shape[1]))
print(X.shape)

(3000, 1, 133)


In [9]:
hp = {
    "in_channels" : 1, 
    "channels": 10, 
    "depth": 10,
    "reduced_size" : 2,
    "out_channels" : 2, 
    "kernel_size": 3,
    "window_length":133,
    "lr": 1e-3, 
    "epochs": 1000,
    "batch_size": 300, 
    "weight_decay":0.0
}